### The  code is designed to predict the outcomes of future football matches for the Corinthians team using historical match data. The main objective is to train a machine learning model that can estimate the probability of Corinthians winning against different opponents. Essentialy what we are doing is:

## Data Collection and Preparation:

 ### Scrape historical match data for Corinthians.
 ### Prepare the data for training a machine learning model by encoding categorical variables and splitting the data into training and testing sets.
## Model Training:
 
 ### Train multiple machine learning models (Random Forest, K-Nearest Neighbors, and Logistic Regression) on the historical data.
 ### Evaluate the models' performance using accuracy scores and validation.
## Prediction:

### Use the trained models to predict the probability of Corinthians winning future matches against specified opponents.


## Visualization:
### Display the predicted win/loss percentages for each model.
### Generate a bar graph to visualize the win percentages against various opponents.

#**Do not for the love of god bet on games based on this code. It is probably not as precise as it seems and I will not be responsible for your financial ruin. The intend is to study how to deal with data, analytics and practice use cases to further my knowledge in Python. Dont be stupid, I love you.**

# Installing Requirements 

In [4]:
!pip install pandas selenium seleniumbase scikit-learn plotly


# Libs

In [5]:
import pandas as pd
from selenium import webdriver
from seleniumbase import Driver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
import plotly.express as px


# Collecting and Prepping Data

In [8]:
dfAnalise = pd.read_csv("jogos.csv")
display(dfAnalise.info())
from sklearn.preprocessing import LabelEncoder as LE
codificador = LE()
for coluna in dfAnalise.columns:
    if dfAnalise[coluna].dtype == object and coluna != "Corinthians_Vitoria":
        dfAnalise[coluna] = codificador.fit_transform(dfAnalise[coluna])
for coluna in dfAnalise.columns:
    if dfAnalise[coluna].dtype == "object" and coluna != "Corinthians_Vitoria":
        print(coluna)
display(dfAnalise.info())


# Drop unnecessary columns and prepare feature matrix (X) and target vector (y)
x = dfAnalise.drop(["vs", "Corinthians_Vitoria", "GolCorinthians ", "GolAdv"], axis=1)
y = dfAnalise["Corinthians_Vitoria"]

# Split data into training and testing sets
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49360 entries, 0 to 49359
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Corinthians          49360 non-null  object
 1   GolCorinthians       49360 non-null  int64 
 2   vs                   49360 non-null  object
 3   GolAdv               49360 non-null  int64 
 4   ADV                  49360 non-null  object
 5   Corinthians_Vitoria  49360 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49360 entries, 0 to 49359
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Corinthians          49360 non-null  int64
 1   GolCorinthians       49360 non-null  int64
 2   vs                   49360 non-null  int64
 3   GolAdv               49360 non-null  int64
 4   ADV                  49360 non-null  int64
 5   Corinthians_Vitoria  49360 non-null  int64
dtypes: int64(6)
memory usage: 2.3 MB


None

# Get to the gym! (training and evaluating models ;) ) 

In [10]:
# Train three different models
modelo_arvore = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()
modelo_LR = LogisticRegression(class_weight="balanced", max_iter=3000, random_state=42)

modelo_arvore.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)
modelo_LR.fit(x_treino, y_treino)

# Evaluate models and choose the best one based on accuracy
previsao_AD = modelo_arvore.predict(x_teste)
previsao_LR = modelo_LR.predict(x_teste)
previsao_KNN = modelo_knn.predict(x_teste)
display(accuracy_score(y_teste, previsao_KNN))
display(accuracy_score(y_teste, previsao_LR))
display(accuracy_score(y_teste, previsao_AD))



0.5754997298757428

0.4952728254997299

0.640194489465154

# Prediction of future matches

In [11]:
# Predict outcomes for new matches
novos_jogos = pd.read_csv('novos_jogos.csv')
novos_jogos = novos_jogos.drop(["vs", "Corinthians_Vitoria", "GolCorinthians ", "GolAdv"], axis= 1)
for coluna in novos_jogos.columns:
    if novos_jogos[coluna].dtype == "object" and coluna != "GolCorinthians ":
        novos_jogos[coluna] = codificador.fit_transform(novos_jogos[coluna])

previsoesA = modelo_arvore.predict(novos_jogos)
previsoesB = modelo_knn.predict(novos_jogos)
previsoesC = modelo_LR.predict(novos_jogos)

# Display predictions
print("Predictions from Random Forest Model (previsoesA):")
print(previsoesA)
print("\nPredictions from KNN Model (previsoesB):")
print(previsoesB)
print("\nPredictions from Logistic Regression Model (previsoesC):")
print(previsoesC)


Predictions from Random Forest Model (previsoesA):
[0 0 1 ... 1 1 1]

Predictions from KNN Model (previsoesB):
[0 1 1 ... 1 1 1]

Predictions from Logistic Regression Model (previsoesC):
[1 0 1 ... 0 0 1]


# Wins and Losses

In [12]:
def display_win_loss_percentages(predictions, model_name):
    total_predictions = len(predictions)
    wins = (predictions == 1).sum()
    losses = (predictions == 0).sum()

    win_percentage = (wins / total_predictions) * 100
    loss_percentage = (losses / total_predictions) * 100

    print(f"\n{model_name} Prediction Percentages:")
    print(f"Win Percentage: {win_percentage:.2f}%")
    print(f"Loss Percentage: {loss_percentage:.2f}%")

display_win_loss_percentages(previsoesA, "Random Forest")
display_win_loss_percentages(previsoesB, "KNN")
display_win_loss_percentages(previsoesC, "Logistic Regression")



Random Forest Prediction Percentages:
Win Percentage: 46.32%
Loss Percentage: 53.68%

KNN Prediction Percentages:
Win Percentage: 54.94%
Loss Percentage: 45.06%

Logistic Regression Prediction Percentages:
Win Percentage: 52.25%
Loss Percentage: 47.75%


# Training and Predicting for Specific Opponents

In [16]:
# Read existing data from jogos.csv
jogos_df = pd.read_csv("jogos.csv")

# Calculate the new column (1 if Corinthians won, 0 otherwise)
jogos_df["Corinthians_Vitoria"] = (jogos_df["GolCorinthians "] > jogos_df["GolAdv"]).astype(int)

# Save the updated DataFrame back to jogos.csv
jogos_df.to_csv("jogos.csv", index=False)

print("Vitorias atualizadas.")

Vitorias atualizadas.


In [17]:
def treinar_e_prever_para_adversario(adversario, jogos_df):
    # Filtrar e preparar os dados
    df_filtrado = jogos_df[(jogos_df["ADV"] == adversario) | (jogos_df["Corinthians"] == adversario)].copy()
    if df_filtrado.empty or df_filtrado["Corinthians_Vitoria"].nunique() <= 1:
        print(f"Dados insuficientes para o adversário: {adversario}")
        return None

    # Codificação de variáveis categóricas
    codificador = LabelEncoder()
    for coluna in df_filtrado.columns:
        if df_filtrado[coluna].dtype == "object" and coluna != "Corinthians_Vitoria":
            df_filtrado[coluna] = codificador.fit_transform(df_filtrado[coluna])

    X = df_filtrado.drop(["vs", "Corinthians_Vitoria", "GolCorinthians ", "GolAdv"], axis=1)
    y = df_filtrado["Corinthians_Vitoria"]

    # Escalonamento e divisão dos dados
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Modelos
    modelos = {
        'KNN': KNeighborsClassifier(n_neighbors=min(5, len(X_treino) - 1)),
        'Logistic Regression': LogisticRegression(class_weight="balanced", max_iter=3000, random_state=42),
        'Random Forest': RandomForestClassifier(class_weight='balanced', min_samples_split=5, random_state=42)
    }

    percentuais_vitoria = {}
    for nome, modelo in modelos.items():
        if len(y_treino.unique()) > 1:  # Verifica se há mais de uma classe
            # Validação Cruzada
            scores = cross_val_score(modelo, X, y, cv=5, scoring='accuracy')
            media_acuracia = scores.mean()
            print(f"\n{nome} para o Adversário: {adversario}")
            print(f"Acurácia Média na Validação Cruzada: {media_acuracia:.2f}")

            # Treinar o modelo e prever a probabilidade
            modelo.fit(X_treino, y_treino)
            previsoes = modelo.predict(X_teste)
            print(f"Acurácia no Teste: {accuracy_score(y_teste, previsoes):.2f}")

            # Dados para previsão
            novos_dados = pd.DataFrame({"Corinthians": [adversario] * len(jogos_df), "ADV": [adversario] * len(jogos_df)})
            for coluna in novos_dados.columns:
                if novos_dados[coluna].dtype == "object":
                    novos_dados[coluna] = codificador.transform(novos_dados[coluna])
            novos_dados = scaler.transform(novos_dados)

            # Previsão de Probabilidade
            probabilidade = modelo.predict_proba(novos_dados)[:, 1]
            percentuais_vitoria[nome] = probabilidade.mean() * 100
        else:
            print(f"Modelo {nome} não treinado para o adversário {adversario} devido à falta de diversidade nas classes.")
            percentuais_vitoria[nome] = None

    print(f"\nPercentuais de Vitória contra {adversario}:")
    for nome, percentual in percentuais_vitoria.items():
        if percentual is not None:
            print(f"{nome}: {percentual:.2f}%")

    return percentuais_vitoria

# Obter a lista de todos os adversários únicos com pelo menos 2 amostras
adversarios_com_suficientes_dados = [
    adv for adv in jogos_df["ADV"].unique()
    if jogos_df[(jogos_df["ADV"] == adv) | (jogos_df["Corinthians"] == adv)]["Corinthians_Vitoria"].nunique() > 1
]

# Visualizing data

In [20]:
# Collect data for the graph
dados_grafico = []
for adversario in adversarios_com_suficientes_dados:
    percentuais_vitoria = treinar_e_prever_para_adversario(adversario, jogos_df)
    if percentuais_vitoria is not None:
        for modelo, percentual in percentuais_vitoria.items():
            if percentual is not None:
                dados_grafico.append({'Adversário': adversario, 'Modelo': modelo, 'Percentual de Vitória': percentual})

# Create the graph
df_grafico = pd.DataFrame(dados_grafico)
fig = px.bar(df_grafico, x='Adversário', y='Percentual de Vitória', color='Modelo',
            title='Percentual de Vitória contra Adversários',
            labels={'Percentual de Vitória': 'Percentual de Vitória (%)', 'Adversário': 'Adversário'},
            barmode='group')
fig.show()



KNN para o Adversário: VASCO DA GAMA
Acurácia Média na Validação Cruzada: 0.46
Acurácia no Teste: 0.43

Logistic Regression para o Adversário: VASCO DA GAMA
Acurácia Média na Validação Cruzada: 0.57
Acurácia no Teste: 0.57


KeyboardInterrupt: 